# Tratamiento de dataset

In [1]:
import cv2
import numpy as np

#import os
#import shutil

import random

from PixelMetro import *

In [25]:
path = './segmentation_masks/'
_, _, filenames = next(walk(path))

files = sorted(sorted(filenames), key=len)
files = random.shuffle(files)[:1000] # Uso las 1000 primeras imagenes

In [26]:
iou_list = []

for name in files:
    numVideo = str(name)[0] # El 1er digito indica num video
    _,_,img = cv2.split(cv2.imread("segmentation_masks/"+name+".png"))
    cap = cv2.VideoCapture('./test_'+numVideo+'.mp4')
    cap.set(cv2.CAP_PROP_POS_FRAMES,int(str(name)[1:-1]))
    ret, frame = cap.read()
    img = cv2.resize(img,(320,180))
    thresh = cv2.threshold(img,128,255,cv2.THRESH_BINARY)[1]
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    for x in range(len(contours)):
        cnt = contours[x]
        x,y,w,h = cv2.boundingRect(cnt)
        cropped = image[x:(w + x), y:(h + y)]
        a,b,c,d = get_Table(cropped)
        trapez = np.copy(cropped) * 0
        ppt = np.array([a,c,d,b], np.int32)
        ppt = ppt.reshape((-1, 1, 2))
        tableDet = cv2.cvtColor(cv2.fillPoly(trapez, [ppt], (255), 8),cv2.COLOR_BGR2GRAY)
        union = cv2.countNonZero(cv2.bitwise_or(tableDet,thresh))
        intersect = cv2.countNonZero(cv2.bitwise_and(tableDet,thresh))
        iou += [intersect/union] 

In [27]:
iou = 0
err = 0

for x in iou_list:
    iou += x
    if x < 0.9:
        err +=1

iou /= len(iou_list)
err /= len(iou_list)

In [32]:
print(iou)

0.912666666666


In [33]:
print(err)

0.072
